In [1]:
"""main wrapper for Non-IPC safety stock reset"""

import os
import sys
import argparse

import pandas as pd
import datetime as dt
from dateutil.tz import gettz
from ast import literal_eval


In [2]:
sys.path.append('../../../..')

In [3]:
from zeno_etl_libs.helper.aws.s3 import S3
from zeno_etl_libs.db.db import DB, PostGre
from zeno_etl_libs.helper import helper
from zeno_etl_libs.logger import get_logger

In [4]:
from zeno_etl_libs.utils.non_ipc.data_prep.non_ipc_data_prep import non_ipc_data_prep
from zeno_etl_libs.utils.non_ipc.forecast.forecast_main import non_ipc_forecast
from zeno_etl_libs.utils.non_ipc.safety_stock.safety_stock import non_ipc_safety_stock_calc
from zeno_etl_libs.utils.warehouse.wh_intervention.store_portfolio_consolidation import stores_ss_consolidation
from zeno_etl_libs.utils.ipc.goodaid_substitution import update_ga_ss
from zeno_etl_libs.utils.ipc.npi_exclusion import omit_npi_drugs
from zeno_etl_libs.utils.ipc.post_processing import post_processing
from zeno_etl_libs.utils.ipc.doid_update_ss import doid_update

/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [5]:
def main(debug_mode, reset_stores, reset_date, type_list, reset_store_ops,
         goodaid_ss_flag, ga_inv_weight, rest_inv_weight, top_inv_weight,
         chronic_max_flag, wh_gen_consolidation, v5_active_flag,
         v6_active_flag, v6_type_list, v6_ptr_cut_off, v3_active_flag,
         omit_npi, corrections_selling_probability_cutoff,
         corrections_cumulative_probability_cutoff, drug_type_list_v4,
         agg_week_cnt, kind, rs_db_read, rs_db_write, read_schema, write_schema,
         logger):

    s3 = S3()
    logger.info(f"Debug Mode: {debug_mode}")
    status = 'Failed'
    if v3_active_flag == 'Y':
        corrections_flag = True
    else:
        corrections_flag = False

    # Define empty DF if required in case of fail
    order_value_all = pd.DataFrame()
    new_drug_entries = pd.DataFrame()
    missed_entries = pd.DataFrame()

    logger.info("Forecast pipeline starts...")
    try:
        for store_id in reset_stores:
            logger.info("Non-IPC SS calculation started for store id: " +
                        str(store_id))

            # RUNNING DATA PREPARATION
            drug_data_agg_weekly, drug_data_weekly, drug_class, \
            bucket_sales = non_ipc_data_prep(
                store_id_list=store_id, reset_date=reset_date,
                type_list=type_list, db=rs_db_read, schema=read_schema,
                agg_week_cnt=agg_week_cnt,
                logger=logger)

            # CREATING TRAIN FLAG TO HANDLE STORES WITH HISTORY < 16 WEEKS
            week_count = drug_data_weekly['date'].nunique()
            if week_count >= 16:
                train_flag = True
            else:
                train_flag = False

            # RUNNING FORECAST PIPELINE AND SAFETY STOCK CALC
            out_of_sample = 1
            horizon = 1
            train, error, predict, ensemble_train, ensemble_error, \
            ensemble_predict = non_ipc_forecast(
                drug_data_agg_weekly, drug_data_weekly, drug_class,
                out_of_sample, horizon, train_flag, logger, kind)

            final_predict = ensemble_predict.query('final_fcst == "Y"')

            safety_stock_df, df_corrections, \
            df_corrections_111, drugs_max_to_lock_ipcv6, \
            drug_rejects_ipcv6 = non_ipc_safety_stock_calc(
                store_id, drug_data_weekly, reset_date, final_predict,
                drug_class, corrections_flag,
                corrections_selling_probability_cutoff,
                corrections_cumulative_probability_cutoff,
                chronic_max_flag, train_flag, drug_type_list_v4,
                v5_active_flag, v6_active_flag, v6_type_list,
                v6_ptr_cut_off, rs_db_read, read_schema, logger)

            # WAREHOUSE GENERIC SKU CONSOLIDATION
            if wh_gen_consolidation == 'Y':
                safety_stock_df, consolidation_log = stores_ss_consolidation(
                    safety_stock_df, rs_db_read, read_schema,
                    min_column='safety_stock', ss_column='reorder_point',
                    max_column='order_upto_point')

            # GOODAID SAFETY STOCK MODIFICATION
            if goodaid_ss_flag == 'Y':
                safety_stock_df, good_aid_ss_log = update_ga_ss(
                    safety_stock_df, store_id, rs_db_read, read_schema,
                    ga_inv_weight, rest_inv_weight,
                    top_inv_weight, substition_type=['generic'],
                    min_column='safety_stock', ss_column='reorder_point',
                    max_column='order_upto_point', logger=logger)

            # OMIT NPI DRUGS
            if omit_npi == 'Y':
                safety_stock_df = omit_npi_drugs(safety_stock_df, store_id,
                                                 reset_date, rs_db_read,
                                                 read_schema, logger)

            # POST PROCESSING AND ORDER VALUE CALCULATION
            safety_stock_df['percentile'] = 0.5
            final_predict.rename(columns={'month_begin_dt': 'date'},
                                 inplace=True)
            drug_class, weekly_fcst, safety_stock_df, \
                order_value = post_processing(
                                        store_id, drug_class, final_predict,
                                        safety_stock_df, rs_db_read,
                                        read_schema, logger)
            order_value_all = order_value_all.append(order_value,
                                                     ignore_index=True)

            # WRITING TO RS-DB
            if debug_mode == 'N':
                logger.info("Writing table to RS-DB")
                # writing table ipc-forecast
                predict['forecast_date'] = dt.datetime.strptime(reset_date, '%Y-%m-%d').date()
                predict['store_id'] = store_id
                predict['month_begin_dt'] = predict['month_begin_dt'].dt.date
                predict['created-at'] = dt.datetime.now(
                    tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
                predict['created-by'] = 'etl-automation'
                predict['updated-at'] = dt.datetime.now(
                    tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
                predict['updated-by'] = 'etl-automation'
                predict.columns = [c.replace('_', '-') for c in predict.columns]
                table_info = helper.get_table_info(db=rs_db_write,
                                                   table_name='non-ipc-predict',
                                                   schema=write_schema)
                columns = list(table_info['column_name'])
                predict = predict[columns]  # required column order

                logger.info("Writing to table: non-ipc-predict")
                s3.write_df_to_db(df=predict,
                                  table_name='non-ipc-predict',
                                  db=rs_db_write, schema=write_schema)

                # writing table non-ipc-safety-stock
                safety_stock_df['reset_date'] = dt.datetime.strptime(reset_date, '%Y-%m-%d').date()
                safety_stock_df['created-at'] = dt.datetime.now(
                    tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
                safety_stock_df['created-by'] = 'etl-automation'
                safety_stock_df['updated-at'] = dt.datetime.now(
                    tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
                safety_stock_df['updated-by'] = 'etl-automation'
                safety_stock_df.columns = [c.replace('_', '-') for c in
                                           safety_stock_df.columns]
                table_info = helper.get_table_info(db=rs_db_write,
                                                   table_name='non-ipc-safety-stock',
                                                   schema=write_schema)
                columns = list(table_info['column_name'])
                safety_stock_df = safety_stock_df[columns]  # required column order

                logger.info("Writing to table: non-ipc-safety-stock")
                s3.write_df_to_db(df=safety_stock_df,
                                  table_name='non-ipc-safety-stock',
                                  db=rs_db_write, schema=write_schema)

                # writing table non-ipc-abc-xyz-class
                drug_class['reset_date'] = dt.datetime.strptime(reset_date, '%Y-%m-%d').date()
                drug_class['created-at'] = dt.datetime.now(
                    tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
                drug_class['created-by'] = 'etl-automation'
                drug_class['updated-at'] = dt.datetime.now(
                    tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
                drug_class['updated-by'] = 'etl-automation'
                drug_class.columns = [c.replace('_', '-') for c in
                                      drug_class.columns]
                table_info = helper.get_table_info(db=rs_db_write,
                                                   table_name='non-ipc-abc-xyz-class',
                                                   schema=write_schema)
                columns = list(table_info['column_name'])
                drug_class = drug_class[columns]  # required column order

                logger.info("Writing to table: non-ipc-abc-xyz-class")
                s3.write_df_to_db(df=drug_class,
                                  table_name='non-ipc-abc-xyz-class',
                                  db=rs_db_write, schema=write_schema)

                # to write ipc v6 tables ...

                # UPLOADING MIN, SS, MAX in DOI-D
                logger.info("Updating new SS to DrugOrderInfo-Data")
                safety_stock_df.columns = [c.replace('-', '_') for c in
                                           safety_stock_df.columns]
                ss_data_upload = safety_stock_df.query('order_upto_point > 0')[
                    ['store_id', 'drug_id', 'safety_stock', 'reorder_point',
                     'order_upto_point']]
                ss_data_upload.columns = ['store_id', 'drug_id', 'corr_min',
                                          'corr_ss', 'corr_max']
                new_drug_entries_str, missed_entries_str = doid_update(
                    ss_data_upload, type_list, rs_db_write, write_schema,
                    logger)
                new_drug_entries = new_drug_entries.append(new_drug_entries_str)
                missed_entries = missed_entries.append(missed_entries_str)

                logger.info("All writes to RS-DB completed!")

                # INTERNAL TABLE SCHEDULE UPDATE - OPS ORACLE
                # logger.info(f"Rescheduling SID:{store_id} in OPS ORACLE")
                # if reset_store_ops != None:
                #     content_type = 74
                #     object_id = reset_store_ops.loc[
                #         reset_store_ops[
                #             'store_id'] == store_id, 'object_id'].unique()
                #     for obj in object_id:
                #         request_body = {
                #             "object_id": int(obj), "content_type": content_type}
                #         api_response, _ = django_model_execution_log_create_api(
                #             request_body)
                #         reset_store_ops.loc[
                #             reset_store_ops['object_id'] == obj,
                #             'api_call_response'] = api_response

            else:
                logger.info("Writing to RS-DB skipped")

        status = 'Success'
        logger.info(f"Non-IPC code execution status: {status}")

    except Exception as error:
        logger.exception(error)
        logger.info(f"Non-IPC code execution status: {status}")

    return order_value_all, new_drug_entries, missed_entries

In [6]:
# Parameter passing

In [7]:
env = "dev"

In [8]:
os.environ['env'] = env

In [9]:
email_to = "vivek.revi@zeno.health"
debug_mode = "N"

# JOB EXCLUSIVE PARAMS
exclude_stores = [52, 60, 92, 243, 281]
goodaid_ss_flag = "Y"
ga_inv_weight = 0.5
rest_inv_weight = 0.0
top_inv_weight = 1
chronic_max_flag = "N"
wh_gen_consolidation = "Y"
v5_active_flag = "N"
v6_active_flag = "N"
v6_type_list = ['ethical', 'generic', 'others']
v6_ptr_cut_off = 400
reset_date = "YYYY-MM-DD"
reset_stores = [263]
v3_active_flag = "N"
corrections_selling_probability_cutoff = "{'ma_less_than_2': 0.40, 'ma_more_than_2' : 0.40}"
corrections_cumulative_probability_cutoff = "{'ma_less_than_2':0.50,'ma_more_than_2':0.63}"
drug_type_list_v4 = "{'generic':'{0:[0,0,0], 1:[0,0,1], 2:[0,1,2],3:[1,2,3]}','ethical':'{0:[0,0,0], 1:[0,0,1], 2:[0,1,2],3:[1,2,3]}','others':'{0:[0,0,0], 1:[0,1,2], 2:[0,1,2],3:[1,2,3]}'}"
agg_week_cnt = 4
kind = 'mae'
omit_npi = 'N'

In [10]:
# EVALUATE REQUIRED JSON PARAMS
corrections_selling_probability_cutoff = literal_eval(
    corrections_selling_probability_cutoff)
corrections_cumulative_probability_cutoff = literal_eval(
    corrections_cumulative_probability_cutoff)
drug_type_list_v4 = literal_eval(drug_type_list_v4)

In [11]:
logger = get_logger()
rs_db_read = DB(read_only=True)
rs_db_write = DB(read_only=False)
read_schema = 'prod2-generico'
write_schema = 'prod2-generico'

In [12]:
# open RS connection
rs_db_read.open_connection()
rs_db_write.open_connection()

In [13]:
if reset_date == 'YYYY-MM-DD':  # Take current date
    reset_date = dt.date.today().strftime("%Y-%m-%d")

In [14]:
if reset_stores == [0]:  # Fetch scheduled Non-IPC stores from OPS ORACLE
    store_query = """
        select "id", name, "opened-at" as opened_at
        from "{read_schema}".stores
        where name <> 'Zippin Central'
        and "is-active" = 1
        and "opened-at" != '0101-01-01 00:00:00'
        and id not in {0}
        """.format(
        str(exclude_stores).replace('[', '(').replace(']', ')'),
        read_schema=read_schema)
    stores = rs_db_read.get_df(store_query)
    # considering reset of stores aged (3 months < age < 1 year)
    store_id = stores.loc[
        (dt.datetime.now() - stores['opened_at'] > dt.timedelta(days=90)) &
        (dt.datetime.now() - stores['opened_at'] <= dt.timedelta(days=365)),
        'id'].values

    # QUERY TO GET SCHEDULED STORES AND TYPE FROM OPS ORACLE
    pg_internal = PostGre(is_internal=True)
    pg_internal.open_connection()
    reset_store_query = """
        SELECT
            "ssr"."id" as object_id,
            "s"."bpos_store_id" as store_id,
            "dc"."slug" as type,
            "ssr"."drug_grade"
        FROM
            "safety_stock_reset_drug_category_mapping" ssr
            INNER JOIN "ops_store_manifest" osm
            ON ( "ssr"."ops_store_manifest_id" = "osm"."id" )
            INNER JOIN "retail_store" s
            ON ( "osm"."store_id" = "s"."id" )
            INNER JOIN "drug_category" dc
            ON ( "ssr"."drug_category_id" = "dc"."id")
        WHERE
            (
                ( "ssr"."should_run_daily" = TRUE OR
                    "ssr"."trigger_dates" && ARRAY[ date('{reset_date}')] )
                AND "ssr"."is_auto_generate" = TRUE
                AND "osm"."is_active" = TRUE
            AND "osm"."is_generate_safety_stock_reset" = TRUE
            AND "dc"."is_safety_stock_reset_enabled" = TRUE
            AND "dc"."is_active" = TRUE
            AND s.bpos_store_id in {store_list}
            )
        """.format(
        store_list=str(list(store_id)).replace('[', '(').replace(']',')'),
        reset_date=reset_date)
    reset_store_ops = pd.read_sql_query(reset_store_query,
                                        pg_internal.connection)
    pg_internal.close_connection()

    reset_store_ops['api_call_response'] = False
    reset_stores = reset_store_ops['store_id'].unique()
    type_list = str(
        list(reset_store_ops.loc[reset_store_ops['store_id'] ==
                                 store_id, 'type'].unique()))
    type_list = type_list.replace('[', '(').replace(']', ')')

else:
    type_list = "('ethical', 'ayurvedic', 'generic', 'discontinued-products', " \
                "'banned', 'general', 'high-value-ethical', 'baby-product'," \
                " 'surgical', 'otc', 'glucose-test-kit', 'category-2', " \
                "'category-1', 'category-4', 'baby-food', '', 'category-3')"
    reset_store_ops = None

In [15]:
""" calling the main function """
order_value_all, new_drug_entries, \
    missed_entries = main(
        debug_mode, reset_stores, reset_date, type_list, reset_store_ops,
        goodaid_ss_flag, ga_inv_weight, rest_inv_weight, top_inv_weight,
        chronic_max_flag, wh_gen_consolidation, v5_active_flag,
        v6_active_flag, v6_type_list, v6_ptr_cut_off, v3_active_flag,
        omit_npi, corrections_selling_probability_cutoff,
        corrections_cumulative_probability_cutoff, drug_type_list_v4,
        agg_week_cnt, kind, rs_db_read, rs_db_write, read_schema,
        write_schema, logger)

2022-05-10 12:41:09,418 - root - INFO - Debug Mode: N
2022-05-10 12:41:09,422 - root - INFO - Forecast pipeline starts...
2022-05-10 12:41:09,423 - root - INFO - Non-IPC SS calculation started for store id: 263
Date range 2019-04-01 2022-05-10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sales_date          305 non-null    object
 1   drug_id             305 non-null    int64 
 2   net_sales_quantity  305 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 7.3+ KB
sales_history: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   attributed_loss_date  1725 non-null   object 
 1   drug_id               1725 non-null   int64  
 2   loss_quantity         1725 

/Users/kuldeep/PycharmProjects/etl/sagemaker-jobs/src/scripts/experiments/../../../../zeno_etl_libs/utils/ipc/data_prep.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal_sales_monthly['week_begin_dt'] = cal_sales_monthly.apply(
/Users/kuldeep/PycharmProjects/etl/sagemaker-jobs/src/scripts/experiments/../../../../zeno_etl_libs/utils/ipc/data_prep.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal_sales_monthly['month_begin_dt'] = cal_sales_monthly.apply(
/Users/kuldeep/PycharmProjects/etl/sage

Distinct no of drugs 7
Distinct dates 217
DF shape 1519
2021-09-27
7
6
30   2022-05-02
29   2022-04-25
28   2022-04-18
27   2022-04-11
Name: date, dtype: datetime64[ns]
7
6
Date range 2019-04-01 2022-05-10
Distinct drug count 5273
No of days 226
removing dates for 2021.0 9.0 39.0
removing dates for 2022.0 5.0 19.0
Distinct no of drugs 5273
Distinct dates 217
DF shape 1144953
2021-09-27
2022-05-10 12:41:48,598 - root - INFO - Previous week date for last 12 weeks2022-02-07 00:00:00
2022-05-10 12:41:48,600 - root - INFO - No net sales of drugs within last 12 weeks0
2022-05-10 12:41:48,600 - root - INFO - Sales of drugs within last 12 weeks6
18
6
  bucket_abc bucket_xyz  drug_id  net_sales  avg_sales_per_drug  sales_cov  \
0          B          X        2  33.000000           16.500000   0.174777   
1          B          Y        3  49.666667           16.555556   0.487712   
2          C          Y        1   1.000000            1.000000   1.000000   

   net_sales_frac  drug_frac  
0    

/Users/kuldeep/PycharmProjects/etl/sagemaker-jobs/src/scripts/experiments/../../../../zeno_etl_libs/utils/non_ipc/data_prep/non_ipc_data_prep.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal_drug_data_classification.rename(
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


2022-05-10 12:41:51,681 - root - ERROR - 'NoneType' object has no attribute 'params'
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 436, in _process_worker
    r = call_item()
  File "/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 288, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/joblib/_parallel_backends.py", line 595, in __call__
    return self.func(*args, **kwargs)
  File "/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/joblib/parallel.py", line 262, in __call__
    return [func(*args, **kwargs)
  File "/Users/kuldeep/miniforge3/envs/mf-ro-p38/lib/python3.8/site-packages/joblib/parallel.py", line 262, in <listcomp>
    return [fun

[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:    2.9s remaining:    5.8s
[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed:    2.9s remaining:    2.9s
[Parallel(n_jobs=4)]: Done   4 out of   6 | elapsed:    2.9s remaining:    1.5s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    2.9s remaining:    0.0s


In [16]:
# close RS connection
rs_db_read.close_connection()
rs_db_write.close_connection()

# SENT EMAIL ATTACHMENTS
logger.info("Sending email attachments..")
# to write ..............

Redshift DB connection closed successfully.
Redshift DB connection closed successfully.
2022-05-10 12:41:51,694 - root - INFO - Sending email attachments..
